In [129]:
# -*-coding = utf-8 -*-
import pdfplumber
import re
import openpyxl
import easygui
import os
from itertools import groupby
import pandas as pd

def writeDangtian(fname):
    pdf = pdfplumber.open(fname)
    # print('开始读取数据')
    filename = 'BALANCE SHEET.txt'
    with open(filename, 'w') as f:
        f.write('')
    with open(filename, 'a') as f:
        for page in pdf.pages:
            content = page.extract_text()
            f.write(content)
    pdf.close()
    return filename

def chuliData(data0):
    kehuhao_rows = []
    for j in range(len(data0)):
        if data0[j].startswith('客户号：'):
            kehuhao_rows.append(j)
    data1 = data0.copy()
    for j in kehuhao_rows:
        data1[j] = 'kehuhao'
    df0 = [['kehuhao']+ list(g) for k, g in groupby(data1, lambda x: x == 'kehuhao') if not k]
    df0 = df0[1:]
    df = df0.copy()
    for j in range(len(df)):
        kehuhao = data0[kehuhao_rows[j]]
        df[j][0] = kehuhao
    return df

def txtToData(filename):
    data0 = []
    with open(filename) as f:
       for readline in f.readlines():
            data0.append(readline.strip())
    return data0

def matchNumber(df8):
    regex_riqi = re.compile(r'日期：(?P<year>\d{4})年(?P<month>\d{2})月(?P<day>\d{2})日')
    regex_jiner = re.compile(r'金额：CNY(\d{1,5}\.\d{1,2})')
    regex_liushuihao = re.compile(r'交易流水号：(\d{6,10}-\d{1,4})')
    regex_name = re.compile(r'(.+手续费) \d{1,5}\.\d{2}')
    # regex_name = re.compile(r'费用名称：(\w+手续费)')
    riqi = '2010 年1月12日'
    jiner = 0
    name = '对公跨行柜台转账汇款手续费'
    liushuihao = '88888888-888'
    result = []
    for hang in df8:
        tmp_hang = []
        for j in hang:
            mat_riqi = regex_riqi.search(j)
            mat_jiner = regex_jiner.search(j)
            mat_liushuihao = regex_liushuihao.search(j)
            mat_name = regex_name.search(j)
            if mat_riqi:
                year = mat_riqi.group('year')
                month = mat_riqi.group('month')
                day = mat_riqi.group('day')
                riqi = year + '/' + str(int(month)) + '/' + str(int(day))
            elif mat_jiner:
                jiner = mat_jiner.group(1)
            elif mat_name:
                name = mat_name.group(1)
            elif mat_liushuihao:
                liushuihao = mat_liushuihao.group(1)
            else:
                continue
        year_month = year + month
        for j in [riqi,jiner,name,liushuihao]:
            tmp_hang.append(j)
        result.append(tmp_hang)
    return result,year_month

def mAres(ws):    #获取合并单元格地址
    m_list = ws.merged_cells
    cr = []
    for m_area in m_list:
        # print(m_area.min_row, m_area.max_row, m_area.min_col, m_area.max_col)
        # 合并单元格的起始行坐标、终止行坐标。。。。，
        r1, r2, c1, c2 = m_area.min_row, m_area.max_row, m_area.min_col, m_area.max_col
        # 纵向合并单元格的位置信息提取出
        if c2 - c1 > 0:
            cr.append((r1, r2, c1, c2))
        else:
            continue

    cr.sort()
    return cr

def unmergeAres(nws,area):
    nws.unmerge_cells(start_row=area[0],start_column=area[2],end_row=area[1],end_column=area[3])

def mergeAres(nws,area):
    nws.merge_cells(start_row=area[0],start_column=area[2],end_row=area[1],end_column=area[3])

def painter(ws,cell):
    j = cell
    #字休
    font_name = j.font.name
    font_size = j.font.size
    font_italic = j.font.italic
    font_bold = j.font.bold
    font_color = j.font.color
    font = openpyxl.styles.Font(name=font_name, size=font_size , bold=font_bold, italic=font_italic, color=font_color)
    #边框
    border_left = j.border.left
    border = openpyxl.styles.Border(left=border_left, right=border_left , top=border_left, bottom=border_left)
    #填充
    fill_type = j.fill.fill_type
    fgColor = j.fill.fgColor
    pattern_fill = openpyxl.styles.PatternFill(fill_type=fill_type, fgColor=fgColor)
    #倾斜
    horizontal = j.alignment.horizontal
    vertical = j.alignment.vertical
    text_rotation = j.alignment.text_rotation
    wrap_text = j.alignment.wrap_text
    alignment = openpyxl.styles.Alignment(horizontal=horizontal, vertical=vertical, text_rotation=text_rotation, wrap_text=wrap_text)
    #数字格式
    number_format = j.number_format
    #protection
    locked = j.protection.locked
    hidden = j.protection.locked
    protection =  openpyxl.styles.protection.Protection(locked=locked,hidden=hidden)
    #行高
    height = ws.row_dimensions[j.row].height
    return font,border,pattern_fill,alignment,number_format,protection,height

def writeToExcel(ws,nws,result,kehuming,shibihao,kemuhao,fapiao,fenshu,suilu,row_start=5):
    for j in range(len(result)):
        riqi = result[j][0]
        jiner = float(result[j][1])
        name = result[j][2]
        liushuihao = result[j][3]
        row = j + row_start
        nws.cell(row, 1).value = j + 1
        nws.cell(row, 2).value = kehuming
        nws.cell(row, 3).value = shibihao
        nws.cell(row, 4).value = kemuhao
        nws.cell(row, 5).value = riqi
        nws.cell(row, 6).value = liushuihao
        nws.cell(row, 7).value = name
        nws.cell(row, 8).value = fapiao
        nws.cell(row, 9).value = fenshu
        nws.cell(row, 10).value = jiner
        nws.cell(row, 11).value = round(jiner / 1.06, 2)
        nws.cell(row, 12).value = suilu
        nws.cell(row, 13).value = jiner - round(jiner / 1.06, 2)

def getStyles(ws,area0):
    styles = []
    for j in area0:
        font,border,pattern_fill,alignment,number_format,protection,height = painter(ws,j)
        style = [font,border,pattern_fill,alignment,number_format,protection,height]
        styles.append(style)
    return styles

def writeStyles(nws,styles,result,max_column,row_start=5):
    len_result = len(result)
    for i in range(row_start,len_result+row_start):
        for j in range(1,max_column+1):
            nws.cell(i,j).font  = styles[j-1][0]
            nws.cell(i,j).border  = styles[j-1][1]
            nws.cell(i,j).fill  = styles[j-1][2]
            nws.cell(i,j).alignment  = styles[j-1][3]
            nws.cell(i,j).number_format  = styles[j-1][4]
            nws.cell(i,j).protection  = styles[j-1][5]
            height  = styles[j-1][6]
        nws.row_dimensions[i].height = height
    return nws

def toEexcel(result,year_month):
    row_start = 5
    msg = '请点选"增值税发票开具申请表.xlsx"'
    fname = easygui.fileopenbox(msg)
    wb = openpyxl.load_workbook(fname)
    ws = wb.active
    ws_name = ws.title
    max_column = ws.max_column
    max_row = ws.max_row
    tail_row_start = max_row - 7
    tail_areas0 = list(ws)[tail_row_start-1:]
    nws = wb.copy_worksheet(ws)
    nws.title = year_month
    kehuming = nws.cell(row_start,2).value
    shibihao = nws.cell(row_start,3).value
    kemuhao = nws.cell(row_start, 4).value
    fapiao = nws.cell(row_start, 8).value
    fenshu = nws.cell(row_start, 9).value
    suilu = nws.cell(row_start, 12).value
    xuhao_col = [j.value for j in nws['A']]
    total_row = max_row - 7
    len_old = total_row  - 5
    m_area = mAres(ws)
    merge_area = [j for j in m_area if j[0] >= total_row]
    for area in merge_area:
        unmergeAres(nws,area)
    if len(result) > len_old:
        nws.insert_rows(total_row, len(result) - len_old)
    elif  len(result) < len_old:
        for j in range(total_row-1,total_row-1-(len_old-len(result)),-1):
            nws.delete_rows(j)
    else :
        pass
    wb.save(fname)
    wb = openpyxl.load_workbook(fname)
    ws = wb[ws_name]
    nws = wb[year_month]
    writeToExcel(ws,nws, result, kehuming, shibihao, kemuhao, fapiao, fenshu, suilu, row_start)
    nws.cell(len(result)+row_start,10).value = '=sum(j5:j{})'.format(len(result)+row_start-1)
    chayi = len(result) - len_old
    new_merge_arer = [(j[0]+chayi,j[1]+chayi,j[2],j[3]) for j in merge_area]
    area0 = ws[row_start]
    styles = getStyles(ws,area0)
    nws = writeStyles(nws,styles,result,max_column,row_start)
    for j in new_merge_arer:
        mergeAres(nws,j)
    wb.save(fname)
    os.startfile(fname)

def main():
    row_start = 5
    msg = '请点选BALANCE SHEET .pdf文件'
    # fname = easygui.fileopenbox(msg)
    fname = r'D:\a00nutstore\fishc\BALANCE SHEET FOR JUNE,2022.pdf'
    path, file = os.path.split(fname)
    os.chdir(path)
    filename = writeDangtian(fname)
    data0 = txtToData(filename)
    print(data0)
    # df = chuliData(data0)
    # df8 = [j for j in df for i in j if '手续费' in i]
    # df9 = df8.copy()
    # nums = set()
    # for j in range(len(df9)):
    #     for i in df9[j]:
    #         if ('支票手续费' in i) or ('退货手续费' in i) or ('退还手续费' in i):
    #             nums.add(j)
    #         else:
    #             continue
    # ys = [df9[num] for num in list(nums)]
    # for j in list(ys):
    #     df9.remove(j)
    # result, year_month = matchNumber(df9)
    # toEexcel(result, year_month)


if __name__ == '__main__':
    main()




['Holiday Inn Riverside Wuhan', 'Balance Sheet', '武汉晴川假日酒店', 'BALANCE SHEET', '资产负债表', '2022年06月', 'As At 2022/006', 'CURRENCY   ( RMB )', 'THIS MONTH LAST MONTH VARIANCE', '本月 上月 差额', 'CURRENT ASSET  流动资金', 'CASH  货币资金', '11010 House Funds 备用金 52,000.00 52,000.00 0.00', '11020 Provnl Inc in House Bank 临时增加的备用金 40,000.00 40,000.00 0.00', 'CASH IN HAND 备用金 B 1 92,000.00 92,000.00 0.00', '11500 Bank Account - 1 银行 - 中行活期 2,812,500.59 1,379,883.51 1,432,617.08', '11510 Bank Account - 2 银行帐户 - 美金 10,084.35 10,084.43 (0.08)', '11520 Bank Account - 3 银行帐户 - 港币 0.00 0.00 0.00', '11530 Bank Account - 4 银行 - 招行活期 0.00 0.00 0.00', '11540 Bank Account - 5 银行 - 招行定期 0.00 0.00 0.00', '11550 Bank Account - 6 银行 - 中行定期 19,018,895.48 19,018,895.48 0.00', '11560 Bank Account - 7 银行帐户 - 建行活期 0.00 0.00 0.00', '11570 Bank Account - 8 银行 - 兴业银行 64,360.55 99,665.86 (35,305.31)', '11580 Bank Account - 9 兴业银行定期 0.00 0.00 0.00', '11590 Bank Account - 10 银行帐户 - 工会 4,028.82 4,205.71 (176.89)', '11600 Bank Accoun

In [130]:
data0 = ['Holiday Inn Riverside Wuhan', 'Balance Sheet', '武汉晴川假日酒店', 'BALANCE SHEET', '资产负债表', '2022年06月', 'As At 2022/006', 'CURRENCY   ( RMB )', 'THIS MONTH LAST MONTH VARIANCE', '本月 上月 差额', 'CURRENT ASSET  流动资金', 'CASH  货币资金', '11010 House Funds 备用金 52,000.00 52,000.00 0.00', '11020 Provnl Inc in House Bank 临时增加的备用金 40,000.00 40,000.00 0.00', 'CASH IN HAND 备用金 B 1 92,000.00 92,000.00 0.00', '11500 Bank Account - 1 银行 - 中行活期 2,812,500.59 1,379,883.51 1,432,617.08', '11510 Bank Account - 2 银行帐户 - 美金 10,084.35 10,084.43 (0.08)', '11520 Bank Account - 3 银行帐户 - 港币 0.00 0.00 0.00', '11530 Bank Account - 4 银行 - 招行活期 0.00 0.00 0.00', '11540 Bank Account - 5 银行 - 招行定期 0.00 0.00 0.00', '11550 Bank Account - 6 银行 - 中行定期 19,018,895.48 19,018,895.48 0.00', '11560 Bank Account - 7 银行帐户 - 建行活期 0.00 0.00 0.00', '11570 Bank Account - 8 银行 - 兴业银行 64,360.55 99,665.86 (35,305.31)', '11580 Bank Account - 9 兴业银行定期 0.00 0.00 0.00', '11590 Bank Account - 10 银行帐户 - 工会 4,028.82 4,205.71 (176.89)', '11600 Bank Account - 11 RMB - 招行积玉桥支行 0.00 0.00 0.00', 'CASH IN BANK 银行存款 B 2 21,909,869.79 20,512,734.99 1,397,134.80', 'TOTAL CASH 货币资金合计 22,001,869.79 20,604,734.99 1,397,134.80', 'ACCOUNT RECEIVABLE  应收帐', '12010 Guest Ledger Control A/C 在住客人控制帐户 342,856.15 3,618,073.06 (3,275,216.91)', '12020 City Ledger Control A/C 公司挂帐控制帐户 4,979,894.17 2,311,864.09 2,668,030.08', '12050 Accounts Due From Owner 应收业主费用 0.00 0.00 0.00', '12090 Other Accounts Receivable 其他应收款 375,457.55 332,077.45 43,380.10', 'ACCTS RECEIVABLE  应收帐 B 3 5,698,207.87 6,262,014.60 (563,806.73)', 'LESS: PROV. FOR BAD DEBT 减：坏帐准备 14,939.68 6,935.59 8,004.09', 'NET ACCOUNT RECEIVABLE 应收帐合计 5,683,268.19 6,255,079.01 (571,810.82)', 'INVENTORIES  存货 B 4', '13010 Food 食品 137,176.00 182,166.02 (44,990.02)', '13020 Beverage 饮料 169,025.28 150,617.14 18,408.14', '13030 Cigarettes 香烟 0.00 0.00 0.00', '13040 Menu & Bar List 菜单及酒水单 0.00 0.00 0.00', '13050 Paper Supplies 纸制品 0.00 0.00 0.00', '13060 Laundry Supplies 洗衣用品 0.00 0.00 0.00', '13070 Medicine 药品 0.00 0.00 0.00', 'SALEABLE MERCHANDISE 销售商品 306,201.28 332,783.16 (26,581.88)', '13110 China & Glassware 瓷器及玻璃器皿 0.00 0.00 0.00', '13120 Silverware/Stainless 银器/不锈钢器皿 0.00 0.00 0.00', 'OPERATING EQUIPMENT 营运设备  0.00 0.00 0.00', '13210 Guest & Cleaning Supplies 客用品及清洁用品 0.00 0.00 0.00', '13220 Fuel Oil 燃料 57,434.51 75,042.48 (17,607.97)', '13230 Engineering&Maintenance Supply 工程及维修 0.00 0.00 0.00', '13240 Printing & Stationery Supplies 印刷及文具 0.00 0.00 0.00', '13250 Give-aways 赠品 0.00 0.00 0.00', 'OPERATING SUPPLIES 营用用品 57,434.51 75,042.48 (17,607.97)', 'TOTAL INVENTORIES 存货合计 363,635.79 407,825.64 (44,189.85)', 'OTHER ACCOUNTS RECEIVABLE 其他流动资金', 'UNEXPIRED INSURANCE 未到期保险 0.00 0.00 0.00', '14030 Prepaid Guest Supplies 预付客用品 0.00 0.00 0.00', '14050 Other Prepaid 其他预付 325,063.90 250,550.83 74,513.07', '14060 Prepaid Subscription 预付订阅 0.00 0.00 0.00', 'PREPAID EXPENSES 预付费用 B 5 325,063.90 250,550.83 74,513.07', 'TRANDE BARTER CONTRACT BARTER 合同   0.00 0.00 0.00', '15030 Other Deposits 其它押金 27,232.00 27,232.00 0.00', 'OTHER CURRENT ASSET 其他流动资金 B 6 27,232.00 27,232.00 0.00TOTAL OTHER CURRENT ASSETS 其他流动资产合计 352,295.90 277,782.83 74,513.07', 'Balance Sheet', 'TOTAL CURRENT ASSETS 流动资产合计 28,401,069.67 27,545,422.47 855,647.20', '', 'LESS: CURRENT LIABILITIES  减：流动负债', 'ACCOUNT PAYABLE - TRADE 应付帐款 2,027,793.36 1,787,369.54 240,423.82', '21020 Other Accounts Payable 其它应付款 1,873,719.80 1,702,570.45 171,149.35', '21040 Telephone & Facsimile Payable 应付电话&传真费 0.00 0.00 0.00', 'ACCOUNT PAYABLE - OTHER 其他应付款 B 7 1,873,719.80 1,702,570.45 171,149.35', 'AMOUNT DUE TO IHG 应计洲际集团总部 108,719.03 154,671.34 (45,952.31)', '21130 IHG Other Payable 洲际其它应付款 0.00 0.00 0.00', '22010 Accrued Payroll 应计工资 0.00 0.00 0.00', '22020 Accrued Pension Insurance 应计养老保险 178,213.92 0.00 178,213.92', '22021 Accrued Medical Insurance 应计医疗保险 0.00 0.00 0.00', '22022 Accrued Unemployment Insurance 应计失业保险 9,653.33 0.00 9,653.33', '22023 Accrued Housing Insurance 应计住房公积金 90,925.60 90,445.60 480.00', '22024 Accrued Injury Insurance 应计工伤保险 1,782.14 0.00 1,782.14', '22025 Accrued Education Fund 应计教育基金 0.00 0.00 0.00', '22026 Accrued Maternity Insurance 应计生育险 0.00 0.00 0.00', '22027 Accrued Telephone 应计电话费 0.00 0.00 0.00', '22028 Accrued Union Fee 应计工会会费 176,039.68 174,291.13 1,748.55', '22030 Accrued Bonus 应计奖金 749,492.26 719,492.26 30,000.00', '22031 Accrued Employee Payroll Taxes 应计员工工资税 19,232.14 16,343.94 2,888.20', '22102 Accrued Audit Fee 应计审计费 151,200.00 142,800.00 8,400.00', '22103 Accrued Staff Passage 应计员工休假交通费 0.00 0.00 0.00', '22105 Accrued Decoration 应计装饰费 0.00 0.00 0.00', '22111 Sundry Other Accrued 应计杂项费用 393,772.58 361,092.50 32,680.08', '22200 Accrued Sales Taxes 应计营业税 0.00 0.00 0.00', '22201 Accrued Other Taxes 应计其他税金 10,969.23 0.03 10,969.20', '22202 Value Added Tax-Output VAT 应交增值税金-销项税 0.00 0.00 0.00', '22203 Value Added Tax-Input VAT 应交增值税金-进项税 0.00 0.00 0.00', '22204 Value Added Tax-Input VAT Clea 应交VAT-进项税核销 (55,848.32) (48,802.21) (7,046.11)', '22205 Value Added Tax-VAT payable 未交增值税金-已交增 58,226.70 (28,272.86) 86,499.56', '22302 Accrued Electricity 应计电费 169,670.40 104,594.40 65,076.00', '22303 Accrued Water 应计水费 12,801.70 6,993.90 5,807.80', 'ACCRUED LIABILITIES 应付债务 B 8 1,966,131.36 1,538,978.69 427,152.67', '23010 Rooms Deposit 客房定金 95,006.98 79,870.98 15,136.00', '23020 Shop Deposit 商店租赁定金 821,400.00 821,400.00 0.00', '23030 Floating Deposit 流动定金 0.00 0.00 0.00', '23031 F&B Deposit 餐饮定金 643,162.62 839,730.30 (196,567.68)', 'ADVANCE DEPOSIT 预缴押金 B 9 1,559,569.60 1,741,001.28 (181,431.68)', '24030 Deferred Income 递延收入 0.00 0.00 0.00', '24100 Cash Sales Clearance 现金对冲科目 0.00 0.00 0.00', '24102 City Ledger Clearance 挂帐对冲科目 0.00 0.00 0.00', '24103 A/R - A/P - Clearance 应收-应付对冲科目 0.00 0.00 0.00', 'OTHER CURRENT LIABILITIES 其他流动负债 B 10 0.00 0.00 0.00', '25010 Provision - Uniform 储备 - 制服 0.00 0.00 0.00', '25011 Provision -Linen 储备 - 布草 0.00 0.00 0.00', '25012 Provision -Silverware 储备 - 银器 0.00 0.00 0.00', '25013 Provision -Chinaware 储备 - 瓷器 0.00 0.00 0.00', '25014 Provision - Menu 储备 - 菜单 0.00 0.00 0.00', 'PROVISION FOR REPLACEMENT 预提营运物资储备 B 11 0.00 0.00 0.00', 'PROVISION FOR CAPITAL REPLACEMENT 资产重置储备 4,250,734.06 4,184,598.76 66,135.30', 'TOTAL CURRENT LIABILITIES 流动负债合计 11,786,667.21 11,109,190.06 677,477.15', 'WORKING CAPITAL 营运资本合计 16,614,402.46 16,436,232.41 178,170.05', 'DEFERRED CHARGES 递沿费用', 'AMORTIZATION 待摊费用 0.00 0.00 0.00', 'TOTAL ASSETS 资产合计 16,614,402.46 16,436,232.41 178,170.05', 'REPRESENTED BY OPERATING CONTROL A/C 业主权益', 'BALANCE BROUGHT FORWARD 未分配利润 237,124,427.51 237,124,427.51 0.00', 'ACCUMULATED EARNING YEAR-TO-DATE 本年利润 (405,922.67) (669,313.28) 263,390.61', 'TOTAL OWNER EQUITIES 业主权益合计 236,718,504.84 236,455,114.23 263,390.61', 'WITHDRAWAL - OWNING COMPANY 业主往来 (220,104,102.38) (220,018,881.82) (85,220.56)', "TOTAL LIABILTES AND OWNER'EQUITY 负债及业主权益合计 16,614,402.46 16,436,232.41 178,170.05", '']
data0

['Holiday Inn Riverside Wuhan',
 'Balance Sheet',
 '武汉晴川假日酒店',
 'BALANCE SHEET',
 '资产负债表',
 '2022年06月',
 'As At 2022/006',
 'CURRENCY   ( RMB )',
 'THIS MONTH LAST MONTH VARIANCE',
 '本月 上月 差额',
 'CURRENT ASSET  流动资金',
 'CASH  货币资金',
 '11010 House Funds 备用金 52,000.00 52,000.00 0.00',
 '11020 Provnl Inc in House Bank 临时增加的备用金 40,000.00 40,000.00 0.00',
 'CASH IN HAND 备用金 B 1 92,000.00 92,000.00 0.00',
 '11500 Bank Account - 1 银行 - 中行活期 2,812,500.59 1,379,883.51 1,432,617.08',
 '11510 Bank Account - 2 银行帐户 - 美金 10,084.35 10,084.43 (0.08)',
 '11520 Bank Account - 3 银行帐户 - 港币 0.00 0.00 0.00',
 '11530 Bank Account - 4 银行 - 招行活期 0.00 0.00 0.00',
 '11540 Bank Account - 5 银行 - 招行定期 0.00 0.00 0.00',
 '11550 Bank Account - 6 银行 - 中行定期 19,018,895.48 19,018,895.48 0.00',
 '11560 Bank Account - 7 银行帐户 - 建行活期 0.00 0.00 0.00',
 '11570 Bank Account - 8 银行 - 兴业银行 64,360.55 99,665.86 (35,305.31)',
 '11580 Bank Account - 9 兴业银行定期 0.00 0.00 0.00',
 '11590 Bank Account - 10 银行帐户 - 工会 4,028.82 4,205.71 (176.

In [138]:
qita = ['LESS: PROV. FOR BAD DEBT','ACCOUNT PAYABLE - TRADE','AMOUNT DUE TO IHG','PROVISION FOR CAPITAL REPLACEMENT','WITHDRAWAL - OWNING COMPANY','BALANCE BROUGHT FORWARD','ACCUMULATED EARNING YEAR-TO-DATE']

In [139]:
qita

['LESS: PROV. FOR BAD DEBT',
 'ACCOUNT PAYABLE - TRADE',
 'AMOUNT DUE TO IHG',
 'PROVISION FOR CAPITAL REPLACEMENT',
 'WITHDRAWAL - OWNING COMPANY',
 'BALANCE BROUGHT FORWARD',
 'ACCUMULATED EARNING YEAR-TO-DATE']

In [160]:
qita_subject = ['12340','21010','21050','25100','29010','29020','29030']

In [161]:
qita_subject_dic = dict(zip(qita,qita_subject))
qita_subject_dic


{'LESS: PROV. FOR BAD DEBT': '12340',
 'ACCOUNT PAYABLE - TRADE': '21010',
 'AMOUNT DUE TO IHG': '21050',
 'PROVISION FOR CAPITAL REPLACEMENT': '25100',
 'WITHDRAWAL - OWNING COMPANY': '29010',
 'BALANCE BROUGHT FORWARD': '29020',
 'ACCUMULATED EARNING YEAR-TO-DATE': '29030'}

In [162]:
def getSubjectCredit(str,qita,qita_subject_dic):
    shuzhi = r'.*\s{1}(?P<credit>\(?\d{1,3}(,\d{3})*\.\d{2}\)?)\s{1}(?:\(?\d{1,3}(,\d{3})*\.\d{2}\)?)\s{1}(?:\(?\d{1,3}(,\d{3})*\.\d{2}\)?)$'
    pattern = r'^(?P<subject>\d{5}).*\s{1}(?P<credit>\(?\d{1,3}(,\d{3})*\.\d{2}\)?)\s{1}(?:\(?\d{1,3}(,\d{3})*\.\d{2}\)?)\s{1}(?:\(?\d{1,3}(,\d{3})*\.\d{2}\)?)$'
    regex_subject_credit = re.compile(pattern)
    mat_subject_credit = regex_subject_credit.search(str)
    if mat_subject_credit:
        sub = mat_subject_credit.group('subject')
        cre = mat_subject_credit.group('credit')
     
    else:
        for i in qita:
            pattern_qita = r'^{}{}'.format(i,shuzhi)
            regex_qita = re.compile(pattern_qita)
            mat = regex_qita.search(str)
            if mat:
                sub = qita_subject_dic[i]
                cre = mat.group('credit')
                break
                
            else :
                sub = ''
                cre = ''
    return sub,cre
           
       
               
    



        

In [163]:
dic = {}
for i in data0:
    sub,cre = getSubjectCredit(i,qita,qita_subject_dic)
    cre = cre.strip()
    cre = cre.replace(',','')
    if sub not in [None,'']:
         if cre.startswith('('):
                cre = cre.replace('(','')
                cre = cre.replace(')','')
                cre = float(cre)*(-1)
                dic[sub] = cre
         else :
                cre = float(cre)
                dic[sub] = cre
    else :
        continue
        

    


In [164]:
dic

{'11010': 52000.0,
 '11020': 40000.0,
 '11500': 2812500.59,
 '11510': 10084.35,
 '11520': 0.0,
 '11530': 0.0,
 '11540': 0.0,
 '11550': 19018895.48,
 '11560': 0.0,
 '11570': 64360.55,
 '11580': 0.0,
 '11590': 4028.82,
 '11600': 0.0,
 '12010': 342856.15,
 '12020': 4979894.17,
 '12050': 0.0,
 '12090': 375457.55,
 '12340': 14939.68,
 '13010': 137176.0,
 '13020': 169025.28,
 '13030': 0.0,
 '13040': 0.0,
 '13050': 0.0,
 '13060': 0.0,
 '13070': 0.0,
 '13110': 0.0,
 '13120': 0.0,
 '13210': 0.0,
 '13220': 57434.51,
 '13230': 0.0,
 '13240': 0.0,
 '13250': 0.0,
 '14030': 0.0,
 '14050': 325063.9,
 '14060': 0.0,
 '15030': 27232.0,
 '21010': 2027793.36,
 '21020': 1873719.8,
 '21040': 0.0,
 '21050': 108719.03,
 '21130': 0.0,
 '22010': 0.0,
 '22020': 178213.92,
 '22021': 0.0,
 '22022': 9653.33,
 '22023': 90925.6,
 '22024': 1782.14,
 '22025': 0.0,
 '22026': 0.0,
 '22027': 0.0,
 '22028': 176039.68,
 '22030': 749492.26,
 '22031': 19232.14,
 '22102': 151200.0,
 '22103': 0.0,
 '22105': 0.0,
 '22111': 39377

In [157]:
from  xlsxlsx import xlsXlsx
def shengji(fname):
    path, filename = os.path.split(fname)
    if os.path.splitext(filename)[1] == '.xls':
        fname = xlsXlsx(fname)
    else:
        fname = fname
    return fname,path

In [177]:
fname_rec = r'D:\a00nutstore\fishc\Reconciliation Jun,2022 - 副本.xls'
fname_rec,path = shengji(fname_rec)



正将excel低版本xls转化为高版本xlsx，请稍等:


In [175]:
print(fname_rec)

D:\a00nutstore\fishc\Reconciliation Jun,2022 - 副本.xlsx


In [179]:
wb = openpyxl.load_workbook(fname_rec)
sheetnames = wb.sheetnames
for key,value in dic.items():
    if int(key)>=12010 and key in sheetnames:
        if key != '21050':
            ws = wb[key]
            ws['k19'].value = value
        else :
            ws = wb['2105&21100']
            ws['k19'].value  = value
wb.save(fname_rec)


['Assets', 'Liabilities&Equity', 'Check list', 'Sample', '11010', '11010&11020-1', '11500', '11500-1', '11550', '11550-1', '11500-1-1', '11510', '11510-1', '11560', '11560-1', '11570', '11570-1', '11590', '11590-1', '12010', '12010-1', '12020', '12020-1', '12030', '12030-1 ', '12050', '12050-1', '12090', '12090-1 ', 'Sheet1', '12090-2', '12100', '12100-1', '12330', '12330-1', '12340', '12340-1 ', '13010', '13010-1', '13020', '13020-1 ', '13220', '13220-1', '13240', '13240-1', '14050', '14050-1', '14050-2', '14070', '14070-1 ', '14080', '14080-1 ', '14080-2', '15030', '15030-1', '15040-2', '21010', '21010-1', '21020', 'RC费用及代扣税款返回', '22021-1 (2)', '21020-1 (2)', '21020-1', '2105&21100', '21050&21100-1', '21050-2', '21100', '21100-1', '21100-2', '21110', '21110-1', '21110-2', '21120', '21120-1 ', '21120-2', '22010', '22010-1', '22020', '22020-11', '22020-1', '22021', '22021-1', '22022', '22022-1 ', '22023', '22023-1', '22024', '22024-1 ', '22025', '22025-1', '22026', '22026-1', '22028', 

In [189]:
sheetnames = ['Assets', 'Liabilities&Equity', 'Check list', 'Sample', '11010', '11010&11020-1', '11500', '11500-1', '11550', '11550-1', '11500-1-1', '11510', '11510-1', '11560', '11560-1', '11570', '11570-1', '11590', '11590-1', '12010', '12010-1', '12020', '12020-1', '12030', '12030-1 ', '12050', '12050-1', '12090', '12090-1 ', 'Sheet1', '12090-2', '12100', '12100-1', '12330', '12330-1', '12340', '12340-1 ', '13010', '13010-1', '13020', '13020-1 ', '13220', '13220-1', '13240', '13240-1', '14050', '14050-1', '14050-2', '14070', '14070-1 ', '14080', '14080-1 ', '14080-2', '15030', '15030-1', '15040-2', '21010', '21010-1', '21020', 'RC费用及代扣税款返回', '22021-1 (2)', '21020-1 (2)', '21020-1', '2105&21100', '21050&21100-1', '21050-2', '21100', '21100-1', '21100-2', '21110', '21110-1', '21110-2', '21120', '21120-1 ', '21120-2', '22010', '22010-1', '22020', '22020-11', '22020-1', '22021', '22021-1', '22022', '22022-1 ', '22023', '22023-1', '22024', '22024-1 ', '22025', '22025-1', '22026', '22026-1', '22028', '22028-1 ', '22030', '22030-1', '22031', '22031-1', '22102', 'Sheet2', '22102-1', '22106-2', '22111', '22111-1', '22111-1 (2)', '22201', '22201-1', '22202', '22202-1 ', '22203', '22203-1 ', '22204', '22204-1', '22204-2', '22205', '22205-1', '22300', '22300-1', '22302', '22302-1', '22303', '22303-1', '23010', '23010-1', '23020', '23020-1', '24010-2', '23030', '23030-1', '24030-2', '23031', '23031-1', '25100', '25100-1', '25100-2', '29010', '29010-1', '29010-2', '29020', '29020-1', '29030', '29030-1', '29030-2', '兼容性报表', '兼容性报表 (1)', '兼容性报表 (2)']
for key,value in dic.items():
    if key == '21050':
        print('2105&21100')
    else :
        if int(key)>=12010 and key in sheetnames:
            print('key',key)
        else :
            continue


key 12010
key 12020
key 12050
key 12090
key 12340
key 13010
key 13020
key 13220
key 13240
key 14050
key 15030
key 21010
key 21020
2105&21100
key 22010
key 22020
key 22021
key 22022
key 22023
key 22024
key 22025
key 22026
key 22028
key 22030
key 22031
key 22102
key 22111
key 22201
key 22202
key 22203
key 22204
key 22205
key 22302
key 22303
key 23010
key 23020
key 23030
key 23031
key 25100
key 29020
key 29030
key 29010


In [183]:
'21050' in dic.keys()

True

In [184]:
int('21050')>=12010

True

In [185]:
'21050' != '21050'

False